1. Write a approx 750-800 (3PAGE) word description of: 
    - Chatbot application (e.g., use-case, domain of operation).
    - Atleast 3 different test cases that clearly illustrate chatbot behavior.
    - How you connected your work to lecture material 
    - Advanced techniques that you used

If you are submitting a PDF, please mention that here. 

-----------------------------------------------------
##### PASTE YOUR TEXT HERE
-----------------------------------------------------

------------
2. CODE
-----------

In [4]:
import requests
from requests.structures import CaseInsensitiveDict
from datetime import date

get_access_token = requests.post('https://id.twitch.tv/oauth2/token?client_id=ch28sul1esy1ul6ye36on4r49f6qxr&client_secret=b0yv81ftmbt2isestd8slt4n7qfd5r&grant_type=client_credentials')
access_token = get_access_token.json()['access_token']

url = "https://api.igdb.com/v4/games"

user_genre = 'Racing'
user_platform = 'PlayStation'
user_vibes = ['ultra-realistic']



headers = CaseInsensitiveDict()
headers["Client-ID"] = "ch28sul1esy1ul6ye36on4r49f6qxr"
headers["Authorization"] = "Bearer " + access_token
headers["grant_type"] = "client_credentials"
headers["Content-Type"] = "application/json"
body = f'''
fields name, genres.name, platforms.name, release_dates.date, rating, summary;
where platforms.name = "{user_platform}" & genres.name = "{user_genre}";
limit 40;
'''
response = requests.post(url, headers=headers, data=body.encode('utf-8'))
data = response.json()


for game in data:
    if 'summary' in game:
        for vibe in user_vibes:
            if vibe in game['summary']:
                game.update({'keyword_count': 1})

games_with_keywords = [game for game in data if 'keyword_count' in game]
sorted_games_with_keywords = sorted(games_with_keywords, key=lambda x: x['keyword_count'], reverse=True)


games_without_keywords = [game for game in data if 'keyword_count' not in game]
sorted_games_without_keywords = sorted(games_without_keywords, key=lambda x: x.get('rating', 0), reverse=True)

sorted_games = sorted_games_with_keywords + sorted_games_without_keywords   

for game in sorted_games:
    result = game['name']
    if 'keyword_count' in game:
        result += ", Keywords matched: " + str(game['keyword_count'])
    if 'rating' in game:
        result += ", Rating: " + str(game['rating'])
    print(result)
    print('\n')

    if 'release_dates' in game:
        if 'date' in game['release_dates'][0]:
            print(date.fromtimestamp(game['release_dates'][0]['date']).year)
    if 'summary' in game:
        print('Summary: ' + game['summary'])
        print('\n')



Destruction Derby, Keywords matched: 1, Rating: 77.58150037180113


1995
Summary: With no rules, no limits and no pit stopping, anything goes in this collision course of crumple zones, hit & run mayhem, and street level slamming where wrecking your opponent's motor in ultra-realistic smashes and crashes is the name of the game. Take on the rigors of a full Championship season and pit your wits against a motley crew of psycho-waster racers such as the Suicide Squaddies, Skum and The Optician as you storm up the rankings from Rookie to Pro.

Reckon you can handle it?

Destruction Derby works with a serial link cable for two player head-to-head action.


F1 Racing Championship, Keywords matched: 1


2000
Summary: You'll line up against the fiercest opponents of all time in this ultimate F-1 simulation. Featuring real circuits from the 1999 Formula One season, F-1 RACING CHAMPIONSHIP boasts the inclusion of the brand new Sepang track in Malaysia. All of the drivers from the ’99 season are 

In [ ]:
from random import Random
import re
import json
import nltk
import requests
from requests.structures import CaseInsensitiveDict
from datetime import date 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')


def load_json():
    with open("chatbot.json") as file:
        data = json.load(file)
    return data

def match_pattern_to_intent(user_input):
    intent = None
    for pattern_to_intent in data["pattern_to_intent"]:
        if re.search(pattern_to_intent["pattern"], user_input):
            intent = pattern_to_intent["intent"]
            break
    return intent
def match_intent_to_response(intent):   
    response = None
    if intent in data["intent_to_response"]:
        responseList = data["intent_to_response"][intent]
        response = responseList[Random().randint(0, len(responseList) - 1)]
    return response
def match_intent_to_action(intent):
    action = None
    if intent in data["intent_to_action"]:
        action = data["intent_to_action"][intent]
    return action
def get_user_confirmation(prompt):
    print("Frank: " + prompt)
    user_input = input("You: ").lower()
    if match_pattern_to_intent(user_input) == "positive":
        return True
    elif match_pattern_to_intent(user_input) == "negative":
        return False
    else:
        return None
def interaction_loop():
    print("Frank: Hello! My name is Frank! How can I help?")
    while True:
        user_input = input("You: ").lower()
        intent = match_pattern_to_intent(user_input)
        action = match_intent_to_action(intent)
        if intent is not None:
            print_response_after_action = True
            if action and action in globals():
                print_response_after_action = globals()[action](user_input)
            if print_response_after_action:
                response = match_intent_to_response(intent)
                if response is not None:
                    print("Frank: " + response.format(**memories)) 
        else:
            print("Frank: " + "I'm sorry, I don't understand that. Can you please rephrase?")
#INTENT FUNCTIONS
def remember_name(input):
    user_name_re = r"my name is|I am|I'm|call me"
    user_name = input[re.search(user_name_re, input).start():] #remove string before regex recongized
    user_name = re.sub(user_name_re, "", user_name).split()[0] #remove regex and split by space, first word is name
    if memories.get("user_name") and memories["user_name"].lower() != user_name:
        if not get_user_confirmation("Frank: Hold up, I thought your name was {}, would you like to change this?".format(memories["user_name"])):
            print("Frank: Okay, I'll stick with that.")
            return False
    memories.update({"user_name": user_name.capitalize()})
    return True


def remember_keywords(user_input):
    custom_stopwords = set(nltk.corpus.stopwords.words('english')).union({"game", "games", "like", "something"})
    user_input = remove_all_patterns(user_input)
    if memories.get("keywords"):
        if not get_user_confirmation("Frank: Sure, I had you down for games with the following vibes: {} Should I update this?".format(", ".join(memories["keywords"]) + ".".capitalize())):
            print("Frank: Okay, I'll stick with that.")
            return False
    tokens = nltk.word_tokenize(user_input)
    print("tokens: " + str(tokens))
    tagged = nltk.pos_tag(tokens)
    keywords = [word for word, pos in tagged if pos in ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'VB', 'VBG'] and word not in custom_stopwords]
    print("keywords: " + str(keywords)) 
    memories.update({"keywords": keywords})
    return True


def remember_platform(user_input):
    platform = None
    for pattern_to_platform in data["pattern_to_platform"]:
        if re.search(pattern_to_platform["pattern"], user_input):
            platform = pattern_to_platform["platform"]
            break
    if platform is not None:
        if memories.get("platform") and memories["platform"] != platform:
            if not get_user_confirmation("Frank: I remember you said you own a {}, would you like to update this?".format(memories["platform"])):
                print("Frank: Okay, I'll stick with that.")
                return False
        memories.update({"platform": platform})
    return True

def remember_genre(user_input):
    genre = None
    for pattern_to_genre in data["pattern_to_genre"]:
        if re.search(pattern_to_genre["pattern"], user_input):
            genre = pattern_to_genre["genre"]
            break
    if genre is not None:
        if memories.get("genre") and memories["genre"] != genre:
            if not get_user_confirmation("Frank: You mentioned you were into {} previously. Should I update this?".format(memories["genre"])):
                print("Frank: Okay, I'll stick with that.")
                return False
        memories.update({"genre": genre})
    return True
    
def recommend_game(user_input):
    remember_platform(user_input)
    remember_genre(user_input)
    remember_keywords(user_input)
    if not memories.get("platform"):
        print("Frank: I'm sorry, I don't know what platform you're on. Can you tell me?")
        remember_platform(input("You: "))
    if not memories.get("genre"):
        print("Frank: I'm sorry, I don't know what genre you're into. Can you tell me?")
        remember_genre(input("You: "))
    get_access_token = requests.post('https://id.twitch.tv/oauth2/token?client_id=ch28sul1esy1ul6ye36on4r49f6qxr&client_secret=b0yv81ftmbt2isestd8slt4n7qfd5r&grant_type=client_credentials')
    access_token = get_access_token.json()['access_token']

    url = "https://api.igdb.com/v4/games"

    user_genre = memories.get("genre")
    user_platform = memories.get("platform")
    user_keywords = memories.get("keywords")

    headers = CaseInsensitiveDict()
    headers["Client-ID"] = "ch28sul1esy1ul6ye36on4r49f6qxr"
    headers["Authorization"] = "Bearer " + access_token
    headers["grant_type"] = "client_credentials"
    headers["Content-Type"] = "application/json"
    body = f'''
    fields name, genres.name, platforms.name, release_dates.date, rating, summary;
    where platforms.name = "{user_platform}" & genres.name = "{user_genre}";
    limit 100;
    '''
    response = requests.post(url, headers=headers, data=body.encode('utf-8'))
    response = response.json()

    for game in response:
        if 'summary' in game:
            for keyword in user_keywords:
                if keyword in game['summary']:
                    game.update({'keyword_count': 1})
    response = [game for game in response if game['name'] not in memories.get('recommended_games', set())]

    games_with_keywords = [game for game in response if 'keyword_count' in game]
    sorted_games_with_keywords = sorted(games_with_keywords, key=lambda x: x['keyword_count'], reverse=True)

    games_without_keywords = [game for game in response if 'keyword_count' not in game]
    sorted_games_without_keywords = sorted(games_without_keywords, key=lambda x: x.get('rating', 0), reverse=True)

    sorted_games = sorted_games_with_keywords + sorted_games_without_keywords   

    print("Frank: I've got some for you, take a look! Say yes if you're interested, or no if you're not.")

    response_list = data["intent_to_response"]["recommend"]
    for game in sorted_games:

        result = game['name']
        if 'keyword_count' in game:
            result += ", Keywords matched: " + str(game['keyword_count'])
        if 'rating' in game:
            result += ", Rating: " + str(game['rating'])
        print(result)

        remember_previously_recommended_title(game['name'])
        if not get_user_confirmation(response_list[Random().randint(0, len(response_list) - 1)].format(name=game['name'])):
            continue
        else:
            describe_game(game)
            break
    return False

def remember_previously_recommended_title(game_name):
    if 'recommended_games' not in memories:
        memories['recommended_games'] = set()
    if game_name not in memories['recommended_games']:
        memories['recommended_games'].add(game_name)

def describe_game(game):
    result = game['name']
    if 'release_dates' in game:
        if 'date' in game['release_dates'][0]:
            result += " ("+str(date.fromtimestamp(game['release_dates'][0]['date']).year)+")"
    if 'summary' in game:
        result += ": " + game['summary']    
    print(result)

def remove_all_patterns(input):
    print("checking for patterns in " + input)
    for pattern in data["pattern_to_intent"] + data["pattern_to_platform"] + data["pattern_to_genre"]:
        input = re.sub(pattern["pattern"], "", input)
    print("result: " + input)
    return input

    
    
memories = {}
data = load_json()
interaction_loop()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Frank: Hello! My name is Frank! How can I help?


You:  hello


Frank: Hi!


You:  my name is bradley


...

tokens: ['me', 'a', 'cute']
keywords: ['cute']
Frank: I've got some for you, take a look! Say yes if you're interested, or no if you're not.
A Hat in Time, Keywords matched: 1, Rating: 84.14692201177559
Frank: How about A Hat in Time?


You:  no


Portal 2, Rating: 91.60456342996625
Frank: I recommend Portal 2. What do you think?


You:  yes


Portal 2 (2011): Sequel to the acclaimed Portal (2007), Portal 2 pits the protagonist of the original game, Chell, and her new robot friend, Wheatley, against more puzzles conceived by GLaDOS, an A.I. with the sole purpose of testing the Portal Gun's mechanics and taking revenge on Chell for the events of Portal. As a result of several interactions and revelations, Chell once again pushes to escape Aperture Science Labs.


In [5]:
# ATLEAST 3 INTERESTING RESULTS - AND FUNCTION CALLS TO GENERATE THEM


3. Reflect on project management (approx 250 words) 
    - Timeliness: Reflect on how consistently you made an effort meet deadlines
    - Organization: Reflect on how managed the code-base as the project size increased

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------


4. Process reflection (approx 200 words) - Discuss the week by week iterative development of your chatbot.
    - Describe each week of chatbot devleopment.
    - What was the feedback you received? How did you work on the feedback to improve chatbot? What new features did you add? 

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------
